In [1]:
import torch

In [2]:
from kobert_hf.kobert_tokenizer import KoBERTTokenizer

In [3]:
from transformers import BertModel

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd

In [5]:
device = torch.device("cuda:7")

In [6]:
train_data = pd.read_csv('./kobert/train_data.csv', encoding='utf-8-sig')

In [7]:
train_dataset = []
for sen, label in zip(train_data['title'], train_data['topic_idx']):
    data_train = []
    data_train.append(sen)
    data_train.append(str(label))
    train_data.append(data_train)

In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))       

In [9]:
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate = 4e-5

In [18]:
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [13]:
tok = nlp.data.BERTSPTokenizer(tokenizer, bertmodel, lower=False)

In [14]:
data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

AttributeError: 'BertModel' object has no attribute 'bos_token'